In [12]:
#
# Downloads, processes data and trains a neural network to identify handwritten digits from MNIST
#
# Nicholas Brawand - nicholasbrawand@gmail.com
#
import matplotlib.pyplot as plt
import tools
import os
import test
import mlp
import numpy as np
import sys

In [14]:

dataUrlList = [
    "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
]

# download data
print("Downloading data")
outFils = []
for l in dataUrlList:
    if not os.path.isfile(os.path.basename(l)):
        outFils.append(tools.DownloadData(l))
    else:
        print(os.path.basename(l), 'already exists')
        outFils.append(tools.DownloadData(l))

# unzip data
imgFils = []
for l in outFils:
    newFil = l.replace('.gz', '')

    if not os.path.isfile(newFil):
        tools.UnzipFil(l, newFil)
    else:
        print(newFil, 'already exists')

    imgFils.append(newFil)

# imgFils are in the following order:
#
# imgFils = 'train-images-idx3-ubyte', 'train-labels-idx1-ubyte',
#           't10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte'

y_train = tools.load_mnist_labels(imgFils[1])
X_train = tools.load_mnist_images(imgFils[0], len(y_train))

y_test = tools.load_mnist_labels(imgFils[3])
X_test = tools.load_mnist_images(imgFils[2], len(y_test))

train-images-idx3-ubyte.gz already exists
train-labels-idx1-ubyte.gz already exists
t10k-images-idx3-ubyte.gz already exists
t10k-labels-idx1-ubyte.gz already exists
train-images-idx3-ubyte already exists
train-labels-idx1-ubyte already exists
t10k-images-idx3-ubyte already exists
t10k-labels-idx1-ubyte already exists


In [6]:
# plot first few images to test import
# test.CreateSamplePlot(X_train, y_train)
# test.PlotSingleKind(X_train, y_train, 25, 4)
# test.PlotSingleKind(X_train, y_train, 25, 7)

In [8]:
network = mlp.NeuralNetMLP(n_output=10,
                       n_features=X_train.shape[1],
                       n_hidden=50,
                       l2=0.1,
                       l1=0.0,
                       epochs=50,
                       eta=0.001,
                       alpha=0.001,
                       decrease_const=0.00001,
                       minibatches=50,
                       shuffle=True,
                       random_state=1)

nn = network.fit(X_train, y_train, print_progress=True)


Epoch: 50/50

In [15]:

plt.plot(range(len(nn.cost_)), nn.cost_)
plt.ylim([0, 2000])
plt.ylabel('Cost')
plt.xlabel('Epochs * 50')
plt.tight_layout()
plt.savefig('../figures/cost.png', dpi=300)


batches = np.array_split(range(len(nn.cost_)), 1000)
cost_ary = np.array(nn.cost_)
cost_avgs = [np.mean(cost_ary[i]) for i in batches]
plt.plot(range(len(cost_avgs)), cost_avgs, color='red')
plt.ylim([0, 2000])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.tight_layout()
plt.savefig('../figures/cost2.png', dpi=300)

y_train_pred = nn.predict(X_train)

if sys.version_info < (3, 0):
    acc = ((np.sum(y_train == y_train_pred, axis=0)).astype('float') / X_train.shape[0])
else:
    acc = np.sum(y_train == y_train_pred, axis=0) / X_train.shape[0]
    print('Training accuracy: %.2f%%' % (acc * 100))

y_test_pred = nn.predict(X_test)

if sys.version_info < (3, 0):
    acc = ((np.sum(y_test == y_test_pred, axis=0)).astype('float') / X_test.shape[0])
else:
    acc = np.sum(y_test == y_test_pred, axis=0) / X_test.shape[0]
    print('Test accuracy: %.2f%%' % (acc * 100))

miscl_img = X_test[y_test != y_test_pred][:25]
correct_lab = y_test[y_test != y_test_pred][:25]
miscl_lab = y_test_pred[y_test != y_test_pred][:25]

fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()

for i in range(25):
    img = miscl_img[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title('%d) t: %d p: %d' % (i+1, correct_lab[i], miscl_lab[i]))

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
plt.savefig('../figures/mnist_miscl.png', dpi=300)





Training accuracy: 90.61%
Test accuracy: 90.97%


/Users/nbrawand/anaconda/lib/python3.5/site-packages/matplotlib/tight_layout.py:222: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
